<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/THESIS_MODEL_V2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages needed**

In [1]:
!pip install gekko
!pip install ipynb


     |████████████████████████████████| 10.8MB 5.9MB/s 


**Importing libraries needed**

In [2]:
import numpy as np
import time
from random import sample
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
from library import *


Mounted at /content/drive


**SAWMILL CLASS**

In [276]:
DISTANCE = 1
DENSITY = 2
RADIO_MILL = 10

class Inventory:
  Total_Residues = []
  def __init__(self, Av, Dist, Wt=-1, Pt=-1):
    self.AcumResidues = []  #Residues generated by harvesting
    self.Available = list()
    self.Available.append(Av.copy()) #Dataframe that contains Available wood Inventory at diferent time t
    self.Available_Mill = [] #Dataframe that contains the Available Inventory close the mill to be harvested
    self.Candidates=[] #Candidate location for biorefinery
    self.RefCost = [] #Matrix Distance from any Mills and candidates cells to other cells
    self.Dist = Dist #Matrix Distance from any Mills and candidates cells to other cells
    self.Dist_Mill = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Dist_Refi = [] #Dataframe that contains the Distance to the mill to be harvested
    #self.Growth = [] #Growth ammount for each hexagon (feet cubic) 
    self.Harvested = [] #Ammount of harvested wood
    self.Harvested_Res = []  #Residues Harvested
    #self.Mills = [] #Dataframe that contains Id, typewood, type_product, and demand of mills
    self.ProdType = Pt #type of product generated (pulp or saw)
    self.Radio_Refinery = 70
    self.Residues = []  #Residues generated by harvesting
    self.WoodType = Wt #type of wood needed (softwood, hardwood)
    self.Residues_Left = 0.30 #portion of residues left in the ground and not used for the biorefinery
    self.By_Mill = []
    #initialize Harvested list similar to available 
    
    Id_li = list(self.Available[0].loc[:,("Unique_ID")])
    r1 = len(self.Available[0])
    Ha_li =[0]*r1
    Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
    self.Harvested.append(pd.DataFrame(Data).copy())
    Data = {'Unique_ID':Id_li,'Growth':Ha_li}
    self.Growth =pd.DataFrame(Data).copy()    
    Data = {'Unique_ID':Id_li}
    self.By_Mill =pd.DataFrame(Data).copy()
    

  def Harvest_Mill(self,t, j,mode=DISTANCE):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    m = GEKKO(remote=True)
    m.options.solver = 3 #IPOPT solver
    m.options.IMODE = 3 #steady state optimization
    Id1 =self.Mills.loc[j,('Unique_ID')] #look for mill in position j
    Name_Mill =self.Mills.loc[j,('Name')] #look for mill in position j
    
    a =self.Cell_Less(Id1,t,RADIO_MILL) #look for cells between the range Radio_Mill
    self.Dist_Mill = a.loc[:, ('DEST','DIST')] #fill Distances to mills
    Num = len(self.Dist_Mill)
    H = m.Array(m.Var,(Num),value = 0)
    Dem = self.Mills.loc[j,('Demand')]
    self.Available_Mill = self.Disp_Radio( Id1, t+1, RADIO_MILL)
    D1 = np.sum(self.Available_Mill)
    print('**Demanda:', Dem, 'Disponible',D1, 'Year: ',t, 'Mill: ',Id1 )
    
    r1 = len(self.Available[0])
    Ha_li =[0]*r1
    self.By_Mill[Name_Mill] = Ha_li
        
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      
    if (mode == DISTANCE):
      prod0 = m.Intermediate(m.sum([self.Dist_Mill.loc[i,('DIST')]*H[i] for i in range(Num)]))    
    elif (mode ==DENSITY):
      prod0 = m.Intermediate(-m.sum([self.Available_Mill[i]*H[i] for i in range(Num)]))

    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=False)
    a = self.Dist_Mill
    flat = [item for l in H for item in l]
    a['Harvested'] = flat  
    ans = a

    #Storing Harvested 
    for j in range(len(ans)):                                                                         
      if not(self.Harvested[t][self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested[t].loc[a2,("Harvested")] = self.Harvested[t].loc[a2,("Harvested")] + ans.loc[j,('Harvested')]  
        self.By_Mill[Name_Mill][a2] = ans.loc[j,('Harvested')] 
      else:
        print(ans.loc[j,('DEST')], 'no hallado')
    
    #Updating Available Inventory
    for j in range(len(ans)):                                                                         
      if not(self.Available[t+1][self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Available[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Available[t+1].loc[a2,("SumOfTotal")] = self.Available[t+1].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')] 
        l0  = self.Available[t+1]["SumOfTotal"]<0
        
  
  def Harvest_All_Mills(self,t,k):
  #Harvest k mills at year t
    print('Stage:',t)
    self.Available.append(self.Available[t].copy())
    for i in range(k):
      print('Cosechando Sawmill:',i)
      self.Harvest_Mill(t,i,DISTANCE)

    self.Residues.append(self.Harvested[t].loc[:,('Unique_ID','Harvested')].copy())
    self.Harvested.append(self.Harvested[t].copy()) 
    self.Harvested[t+1].loc[:,('Harvested')] = 0
    self.Residues[t].loc[:,('Harvested')] = round(self.Residues[t].loc[:,('Harvested')]*0.05)
    
  
  def Harvest_All_Mills_All_Years(self,Y):
    N_Mills = len(self.Mills)
    for y in range(YEARS):
      self.Harvest_All_Mills(y,1) 


  def Inventory_Growth(self,t,rate = 0):
    #increase the available inventory of period t
    if (rate == 0):
      r = self.Growth
    else:
      r = rate
    self.Available[t].loc[:,("SumOfTotal")] = self.Available[t].loc[:,("SumOfTotal")]*(1+r/100)
  




  def Growth_Update(self,Data):
    N = len(self.Growth)
    print('Numero de Filas', N)
    if self.ProdType == 'Sawmill':
      cl = 1
    elif self.ProdType == 'Pulpmill':
      cl =2
    
    Data = Data[Data['CLASS'] == cl]
    
    for i in range(N):    
      Id = self.Growth.loc[i,("Unique_ID")]
      Row = Data[Data['Unique_ID'] == Id]
      if not Row.empty:
        if self.WoodType == 'H':
          self.Growth.loc[i,('Growth')] = Row.iloc[0,4] + Row.iloc[0,5] + 0.33 * Row.iloc[0,8]
        if self.WoodType == 'S':
          self.Growth.loc[i,('Growth')] = Row.iloc[0,6] + Row.iloc[0,7] + 0.67 * Row.iloc[0,8]
    




  def Cell_Less(self, Id_Mill, t, Radio=  RADIO_MILL):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Availability
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal.copy()))
    Ans["AVAIL"] = Disp 
    return(Ans)

  def Total_Available(self, Id_Mill, Radio=RADIO_MILL):
    #Given a Radio and a Mill ID this function returns a the total inventory Available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    
    for j in range(len(Ans)):
      if self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[0][self.Available[0]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal.copy()))
    Total = np.sum(Disp)
    return(Total)


  def Cell_Less_Res(self, Id_Mill, t, Radio=50):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Forest Residues available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= self.Radio_Refinery)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]

    for j in range(len(Ans)):                                                                         
      if self.Residues[t][self.Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Residues[t][self.Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].Harvested))
    Ans["AVAIL"] = Disp 
    return(Ans)


  def Disp_Radio(self, Id_Mill, t, Radio=RADIO_MILL):
    #Given a Time and Radio and a Mill ID this function returns a list of Inventory Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):                                                                         
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    return(Disp)
  

  def Num_Cell(self, Id_Mill, Radio=RADIO_MILL):
    #Given a Radio and a Mill ID this function returns cell among the radio distance
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)]
    return(len(Ans))
  
  def Mills_Update(self,M):
    i1 = (M['s_h'] == self.WoodType)&(M['plant'] == self.ProdType)
    self.Mills = M.loc[i1,:]
    Num = len(self.Mills)
    ind = [i for i in range(Num)]
    self.Mills.index = ind
    

  def Disp_Radio_Res(self, Id_Mill, t, Radio=20):
    #Given a Time and Radio and a Mill ID this function returns a list of Residues Available [Unique_ID, Amount_Available]
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= self.Radio_Refinery)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    #update index from 0 to rows-1
    Disp =[]
    for j in range(len(Ans)):  
      #print(self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')])                                                   
      if self.AcumResidues[t][self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.AcumResidues[t][self.AcumResidues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal))
    return(Disp)


  def Harvest_Refinery(self,t,Cand, Demanda=20000): #Evaluates Candidate location for refinery in year t
    m2 = GEKKO(remote=True)
    m2.options.solver = 3 #IPOPT solver
    m2.options.IMODE = 3 #steady state optimization

    Id1 =self.Candidates[Cand] #assign unique ID
    a =self.Cell_Less_Res(Id1,t,self.Radio_Refinery) #look for hexagons between radio distance
    self.Dist_Refi = a.loc[:, ('DEST','DIST')]  #obtain column Destination and Distance from 'a'
    Num = len(self.Dist_Refi) #calculate the number of cells betwee 
    H = m2.Array(m2.Var,(Num),value = 0)

    self.Available_Refi = self.Disp_Radio_Res(Id1, t+1, self.Radio_Refinery) #look for available quantity in cells
      
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Refi[i]  
    prod0 = m2.Intermediate(m2.sum([self.Dist_Refi.loc[i,('DIST')]*H[i] for i in range(Num)]))
    m2.Equation(Demanda ==m2.sum([H[i] for i in range(0,len(H))]))
    m2.Obj(prod0) 
    m2.solve(disp=False,debug=False)
    
    a = self.Dist_Refi
    flat = [item for l in H for item in l]
    a['Harvested'] = flat
    ans = a
        
    #Storing Harvested 
    for j in range(len(ans)):                                                                         
      if not(self.Harvested_Res[t][self.Harvested_Res[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested_Res[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested_Res[t].loc[a2,("Harvested")] = self.Harvested_Res[t].loc[a2,("Harvested")] + ans.loc[j,('Harvested')]  
    
    #Updating Available Inventory
    print('Residues Available Before',np.sum(self.AcumResidues[t+1].loc[:,("SumOfTotal")]))     
    
    for j in range(len(ans)):                                                                         
      if not(self.AcumResidues[t+1][self.AcumResidues[t+1]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.AcumResidues[t+1]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.AcumResidues[t+1].loc[a2,("SumOfTotal")] = self.AcumResidues[t+1].loc[a2,("SumOfTotal")] - ans.loc[j,('Harvested')] 
    
        l0  = self.AcumResidues[t+1]["SumOfTotal"]<0
        if np.sum(self.Available[t+1].loc[:,("SumOfTotal")])<0 :
          print('****Available Negative')
    
    print('answer\n',ans)
    print('Cells Harvested',self.Harvested_Res[t][self.Harvested_Res[t]["Harvested"]>0]) 
    print('Distances',self.Dist_Refi.loc[self.Harvested_Res[t]["Harvested"]>0],('DIST')) 
    print('Total Harvested',np.sum(self.Harvested_Res[t].loc[:,("Harvested")])) 
    print('Residues Available After',np.sum(self.AcumResidues[t+1].loc[:,("SumOfTotal")])) 
    res = round(np.dot(flat,self.Dist_Refi.loc[:,('DIST')]))  #Delivery cost to refinery
    self.RefCost[Cand] += res
    print('Cost', self.RefCost[Cand])
  

  def Optimal_Location_Total(self,Y,NCan = 0):
    #run Ncan locations from year 0 to year 'y'
    if (NCan == 0):
      Cans = len(self.Candidates)
    else:
      Cans = NCan
    self.RefCost = [0]*Cans

    for c in range(Cans):
      self.AcumResidues = []
      self.AcumResidues.append(self.Residues[0].copy())
      self.AcumResidues[0].columns=["Unique_ID", "SumOfTotal"]
      self.AcumResidues[0].loc[:,('SumOfTotal')] = 0
        
      for i in range(Y):
        self.AcumResidues.append(self.AcumResidues[i].copy())
        self.AcumResidues[i+1].columns=["Unique_ID", "SumOfTotal"]
        self.AcumResidues[i+1]['SumOfTotal'] =self.AcumResidues[i+1]['SumOfTotal'] + self.Residues[i]['Harvested']
        self.Harvest_Refinery(i,c)
        self.Harvested_Res.append(self.Harvested_Res[i].copy())     
        self.Harvested_Res[i+1].loc[:,('Harvested')] = 0


  def Williness_Harvest(self, Factor=0):
    l = len(self.Residues[0])
    c = int(l*Factor)
    No_Williness = random.sample(range(l), c)
    retung(No_Willines)
    
      
 
    

**UPLOADING FOREST INVENTORY DATA**

**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**

In [271]:
# Main Program
start_time = time.time()
#Upload distances from each hexagon to other
Hardwood_Saw_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Saw_V3.csv")
Hardwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Pulp_V3.csv")
Softwood_Saw_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Saw_V3.csv")
Softwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Pulp_V3.csv")
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Dist_Mill.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills_TN_V2.csv")
Growth_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Growth_Hexagon.csv")
Aux = Growth_Data[(Growth_Data['OWNERSHIP']=='PRIVATE')&(Growth_Data['SLOPE']=='LE40')] #FILTER OWNERSHIP AND SLOPE
Growth_Data = Aux
Growth_Data = Growth_Data.fillna(0)

#Applying factors to have all the data in cubic feet
i1 = Mills_Data['plant'] == "Composite Panel"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*128 #transforming cords to cubic feet
i1 = Mills_Data['plant'] == "Pulpmill"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*128 #transforming cords to cubic feet
i1 = Mills_Data['plant'] == "Sawmill"
Mills_Data.loc[i1,('Demand')] = Mills_Data.loc[i1,('Demand')]*83.3  #transforming mbf to cubic feet
i1 = Mills_Data['plant'] == "Venner"
Mills_Data.loc[i1,('Demand')] = 133280 #transforming mbf to cubic feet

Mills_Data[Mills_Data.Unique_ID==86399] #removing this mills with no data
Mills_Data.drop(index=106, inplace=True)
Mills_Data[Mills_Data.Unique_ID==86639] #removing this mills with no data
Mills_Data.drop(index=3, inplace=True)

YEARS = 1

#Creating Inventories Objects and Initializing
Hardwood_Saw = Inventory(Hardwood_Saw_Data,Dist_Data,Wt='H',Pt ='Sawmill')
Hardwood_Pulp = Inventory(Hardwood_Pulp_Data,Dist_Data,Wt='H',Pt ='Pulpmill')
Softwood_Saw = Inventory(Softwood_Saw_Data,Dist_Data,Wt='S',Pt ='Sawmill')
Softwood_Pulp = Inventory(Softwood_Saw_Data,Dist_Data,Wt='S',Pt ='Pulpmill')
Hardwood_Pulp.Growth_Update(Growth_Data)

#updating Mills According the wood type and product type
Hardwood_Saw.Mills_Update(Mills_Data)
Hardwood_Pulp.Mills_Update(Mills_Data)
Softwood_Saw.Mills_Update(Mills_Data)
Softwood_Pulp.Mills_Update(Mills_Data)



#Harvest Inventory Objects
Hardwood_Saw.Harvest_All_Mills_All_Years(YEARS)


for y in range(YEARS):
  Hardwood_Saw.Harvested[y].to_csv('/content/drive/My Drive/Colab Notebooks/Results/year'+str(y)+'.csv')
  Hardwood_Saw.By_Mill.to_csv('/content/drive/My Drive/Colab Notebooks/Results/By_Mill'+str(y)+'.csv')
  


#Totalize the Wood Harvested 
Cs =len(Hardwood_Saw.Harvested[YEARS])
for y in range(YEARS):
  for i in range(Cs):
    Hardwood_Saw.Harvested[YEARS].loc[i,('Harvested')] = Hardwood_Saw.Harvested[YEARS].loc[i,('Harvested')] + Hardwood_Saw.Harvested[y].loc[i,('Harvested')] 

Hardwood_Saw.Harvested[YEARS].to_csv('/content/drive/My Drive/Colab Notebooks/Results/year'+str(YEARS)+'.csv')

print ("*********1ST STAGE TOOK", time.time() - start_time, "TO RUN")

Numero de Filas 27180
Id No Encontrado:  95474
Id No Encontrado:  75087
Id No Encontrado:  78415
Id No Encontrado:  82664
Id No Encontrado:  71319
Id No Encontrado:  78414
Id No Encontrado:  83344
Id No Encontrado:  71767
Id No Encontrado:  75518
Id No Encontrado:  72425
Id No Encontrado:  95717
Id No Encontrado:  66346
Id No Encontrado:  69796
Id No Encontrado:  79526
Id No Encontrado:  76636
Id No Encontrado:  71537
Id No Encontrado:  68071
Id No Encontrado:  70018
Id No Encontrado:  75965
Id No Encontrado:  94980
Id No Encontrado:  71544
Id No Encontrado:  67422
Id No Encontrado:  79082
Id No Encontrado:  73308
Id No Encontrado:  66343
Id No Encontrado:  83815
Id No Encontrado:  94737
Id No Encontrado:  70888
Id No Encontrado:  77515
Id No Encontrado:  71331
Id No Encontrado:  71536
Id No Encontrado:  76195
Id No Encontrado:  76409
Id No Encontrado:  67852
Id No Encontrado:  78406
Id No Encontrado:  73754
Id No Encontrado:  69585
Id No Encontrado:  75752
Id No Encontrado:  72860
Id 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


**Demanda: 8330.0 Disponible 72547022 Year:  0 Mill:  65748
*********1ST STAGE TOOK 47.08267164230347 TO RUN


**SECOND STAGE**

In [272]:
start_time = time.time()
#Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
#Hardwood_Saw.Harvested_Res.append(pd.DataFrame(Data).copy())
#Hardwood_Saw.Optimal_Location_Total(YEARS,2)

print ("*********2ND STAGE TOOK", time.time() - start_time, "TO RUN")

*********2ND STAGE TOOK 3.910064697265625e-05 TO RUN
